In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab1.ipynb")

![](img/563_lab_banner.png)

# Lab 1: Document clustering 

## Imports <a name="im"></a>

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
pd.set_option("display.max_colwidth", 0)

<br><br><br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## Submission instructions <a name="si"></a>
rubric={mechanics}

You will receive marks for correctly submitting this assignment by following the instructions below:
    
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).
- [Here](https://github.com/UBC-MDS/public/tree/master/rubric) you will find the description of each rubric used in MDS.
- Make at least three commits in your lab's GitHub repository.    
- Push the final .ipynb file with your solutions to your GitHub repository for this lab.        
- Before submitting your lab, run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).     
- Make sure to enroll to Gradescope via [Canvas](https://canvas.ubc.ca/courses/106525).
- Upload the .ipynb file to Gradescope.
- Make sure that your plots/output are rendered properly in Gradescope.    
- If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf (preferably WebPDF) or html export of .ipynb file with your solutions so that TAs can view your submission on Gradescope. 
- The data you download for this lab <b>SHOULD NOT BE PUSHED TO YOUR REPOSITORY</b> (there is also a `.gitignore` in the repo to prevent this).
- Include a clickable link to your GitHub repo for the lab just below this cell.
</div>    

_Points:_ 3

YOUR REPO LINK GOES HERE

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1: Document clustering warm-up
<hr>

In the lecture we explored image clustering. In this lab, you will explore another popular application of clustering, [**document clustering**](https://en.wikipedia.org/wiki/Document_clustering). A large amount of unlabeled text data is available out there (e.g., news, recipes, online Q&A, and tweets). Clustering is a commonly used technique to organize this data in a meaningful way. 

As a warm up, in this exercise, you will cluster sentences from a toy corpus. Later in the lab, you will work with a more realistic corpus. 

**Your tasks:**

Run the code below which 
- extracts content of Wikipedia articles on a set of queries and stores the first sentence in each article as a document representing that topic,
- tokenizes the text (i.e., separates "words" in the sentence), and 
- carries out preliminary preprocessing (e.g., removes punctuation marks and converts the text to lower case).

> *Note 1: Typically, text preprocessing or normalization is an elaborate process before carrying out document clustering. But in this lab we'll carry out minimal preprocessing, as we will be dealing with fairly short documents which are more or less clean.* 

> *Note 2: If you have created `conda` environment using [the course `yml` file](https://github.ubc.ca/mds-2022-23/DSCI_563_unsup-learn_students/blob/master/env-dsci-563.yml), you should have the following packages. If not, you may have to install appropriate packages in our course's environment.*

> *Note 3: Feel free to experiment with Wikipedia queries of your choice. But stick to the provided list for the final submission so that it's easier for the TAs to grade your submission.*

> *Note 4: For tokenization we are using the `nltk` package. Even if you have the package installed via the course `conda` environment, you might have to download `nltk` pre-trained models, which can be done with the code below.*

In [ ]:
import nltk

nltk.download("punkt")

In [ ]:
import wikipedia
import string 

from nltk.tokenize import sent_tokenize, word_tokenize

queries = [
    "baguette food",
    "banana bread food",
    "bread food",
    "data science",
    "sports analytics",
    "football sport",
    "ice hockey",
]

wiki_dict = {"wiki query": [], "text": [], "n_words": []}
remove_tokens = list(string.punctuation)
remove_tokens.extend(['``','’', '`','br','"',"”", "''", "'s", "(", ")", "[","]"])   
        
for i in range(len(queries)):
    text = sent_tokenize(wikipedia.page(queries[i]).content)[0]    
    tokenized = word_tokenize(text)
    text_pp = [token.lower() for token in tokenized if token.lower() not in remove_tokens]
    wiki_dict["n_words"].append(len(text_pp))
    wiki_dict["text"].append(" ".join(text_pp))    
    wiki_dict["wiki query"].append(queries[i])

wiki_df = pd.DataFrame(wiki_dict)
wiki_df

Our toy corpus has seven toy documents (`text` column in `wiki_df`) extracted from seven Wikipedia queries (`wiki query` column in `wiki_df`). 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many clusters? 
rubric={reasoning}


**Your tasks:**

1. If you are asked to manually cluster the documents from this toy corpus, how many clusters would you identify and how would you label each cluster?   

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 1

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 K-Means with bag-of-words representation 
rubric={accuracy}

In the lecture, we saw that data representation plays a crucial role in clustering. Changing flattened representation of images to feature vectors extracted from pre-trained models greatly improved the quality of clustering. 

What kind of representation is suitable for text data? In previous machine learning courses, we have used bag-of-words representation to numerically encode text data, where each document is represented with a vector of word frequencies. 

Let's try clustering documents with this simplistic representation.  

**Your tasks:**

1. Create bag-of-words representation using [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) with default arguments for the `text` column in `wiki_df` above.
2. Cluster the encoded documents with [`KMeans` clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). Use `random_state=42` (for reproducibility) and set `n_clusters` to the number you identified in the previous exercise. 

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 3

In [ ]:
...

In [ ]:
kmeans_bow_labels = ...

In [ ]:
wiki_df["bow_kmeans"] = kmeans_bow_labels
wiki_df

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 K-Means with sentence embedding representation
rubric={accuracy}

Bag-of-words representation is limited in that it does not take into account word ordering and context. There are other richer and more expressive representations of text which can be extracted using transfer learning. In this lab, we will use one such representation called **sentence embedding representation**, which uses deep learning models to generate dense, fixed-length vector representations for sentences. We will extract such representations using [sentence transformer](https://www.sbert.net/index.html) package. Sentence embedding takes into account context of words and semantic meaning of sentences and it is likely to work better when we are interested in clustering sentences based on their semantic similarity. We'll learn more about these representations later in this course and in DSCI 575 if we get a chance.

**Your tasks:**

1. Run the code below to create sentence embedding representation of documents in our toy corpus. 
2. Cluster documents in our toy corpus encoded with this representation (`emb_sents`) and `KMeans` with following arguments: 
    - `random_state=42` (for reproducibility)
    - `n_clusters`=the number of clusters you identified in 1.1   

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("paraphrase-distilroberta-base-v1")

In [ ]:
emb_sents = embedder.encode(wiki_df["text"]).tolist()
emb_sent_df = pd.DataFrame(emb_sents, index=wiki_df.index)
emb_sent_df

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 2

In [ ]:
...

In [ ]:
kmeans_emb_labels = ...

In [ ]:
wiki_df["emb_kmeans"] = kmeans_emb_labels
wiki_df

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 DBSCAN with sentence embedding representation and cosine distance  
rubric={accuracy}

Now try [`DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) on our toy dataset. K-Means is kind of bound to the Euclidean distance because it is based on the notion of means. With `DBSCAN` we can try different distance metrics. In the context of text data, [cosine similarities](https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity) or cosine distances tend to work well. Given vectors $u$ and $v$, the **cosine distance** between the vectors is defined as: 

$$distance_{cosine}(u,v) = 1 - (\frac{u \cdot v}{\left\lVert u\right\rVert_2 \left\lVert v\right\rVert_2})$$


**Your tasks**

1. Cluster documents in our toy corpus encoded with sentence embedding representation (`emb_sents`) and [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html?highlight=dbscan#sklearn.cluster.DBSCAN) with `metric='cosine'`. You will have to set appropriate values for the hyperparamters `eps` and `min_samples` to get meaningful clusters, as default values of these hyperparameters are unlikely to work well on this toy dataset.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

In [ ]:
...

In [ ]:
dbscan_emb_labels = ...

In [ ]:
wiki_df["emb_dbscan"] = dbscan_emb_labels
wiki_df

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Hierarchical clustering with sentence embedding representation
rubric={accuracy}

**Your tasks:**

Try hierarchical clustering on `emb_sents`. In particular
1. Create and show a dendrogram with `complete` linkage and `metric='cosine'` on this toy dataset.
2. Create flat clusters using `fcluster` with appropriate hyperparameters and store cluster labels to `hier_emb_labels` variable below.

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 3

In [ ]:
...

In [ ]:
hier_emb_labels = ...

In [ ]:
wiki_df["emb_hierarchical"] = hier_emb_labels
wiki_df

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={reasoning}

**Your tasks:**

1. Reflect on and discuss the clustering results of the methods you explored in the previous exercises, focusing on the following points:    
    - effect of input representation on clustering results
    - whether the clustering results match with your intuitions and the challenges associated with getting the desired clustering results with each method

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.7 Visualizing clusters
rubric={viz}

One thing we can do with unlabeled data is visualizing it. That said, our data is high dimensional and high-dimensional data is hard to visualize. For example, in the sentence embedding representation, each example is represented with 768 dimensions. One way to visualize high-dimensional data is by applying dimensionality reduction to get the most important (2 or 3) components of the dataset and visualizing this low-dimensional data. 

Given data as a `numpy` array and corresponding cluster assignments, the `plot_umap_clusters` function below transforms the data by applying dimensionality reduction technique called [UMAP](https://umap-learn.readthedocs.io/en/latest/) to it and plots the transformed data with different colours for different clusters. 

> *Note: At this point we are using this function only for visualization and you are not expected to understand the UMAP part. We'll learn about similar dimensionality reduction techniques in the next two weeks.* 

**Your tasks:**

1. Visualize the clusters created by the methods above using `plot_umap_clusters` function below. In other words, visualize clusters identified by each of the methods below. 
    - K-Means with bag-of-words representation 
    - K-Means with sentence embedding representation
    - DBSCAN with sentence embedding representation 
    - Flat cluster of hierarchical clustering with sentence embedding representation     


In [ ]:
import umap

def plot_umap_clusters(
    data,
    cluster_labels,
    raw_sents=wiki_df["text"],
    show_labels=False,
    size=50,
    n_neighbors=15,
    title="UMAP visualization",
    ignore_noise=False,
):
    """
    Carry out dimensionality reduction using UMAP and plot 2-dimensional clusters.

    Parameters
    -----------
    data : numpy array
        data as a numpy array
    cluster_labels : list
        cluster labels for each row in the dataset
    raw_sents : list
        the original raw sentences for labeling datapoints
    show_labels : boolean
        whether you want to show labels for points or not (default: False)
    size : int
        size of points in the scatterplot
    n_neighbors : int
        n_neighbors hyperparameter of UMAP. See the documentation.
    title : str
        title for the visualization plot

    Returns
    -----------
    None. Shows the clusters.
    """

    reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=42)
    Z = reducer.fit_transform(data)  # reduce dimensionality
    umap_df = pd.DataFrame(data=Z, columns=["dim1", "dim2"])
    umap_df["cluster"] = cluster_labels

    if ignore_noise:
        umap_df = umap_df[umap_df["cluster"] != -1]

    labels = np.unique(umap_df["cluster"])

    fig, ax = plt.subplots(figsize=(6, 5))
    ax.set_title(title)

    scatter = ax.scatter(
        umap_df["dim1"],
        umap_df["dim2"],
        c=umap_df["cluster"],
        cmap="tab20b",
        s=size,
        #edgecolors="k",
        #linewidths=0.1,
    )

    legend = ax.legend(*scatter.legend_elements(), loc="best", title="Clusters")
    ax.add_artist(legend)

    if show_labels:
        x = umap_df["dim1"].tolist()
        y = umap_df["dim2"].tolist()
        for i, txt in enumerate(raw_sents):
            ax.annotate(" ".join(txt.split()[:10]), (x[i], y[i]))
    plt.show()

<div class="alert alert-warning">

Solution_1.7
    
</div>

_Points:_ 3

In [ ]:
...

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: [Food.com](https://www.food.com/) recipes 
<hr>

Now that we have applied document clustering on a toy corpus, let's move to a more realistic corpus. 

In the lecture, we worked on an activity of manually clustering food items and discussed challenges associated with it. We also applied different clustering algorithms to cluster food images. We'll continue this theme of clustering food items in this lab. But instead of images we will cluster textual description of food items, i.e., recipe names.   

In this lab, we will work with a sample of [Kaggle's Food.com recipes corpus](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions). This corpus contains 180K+ recipes and 700K+ recipe reviews. In this lab, we'll only focus on recipes and **not** on reviews. The recipes are present in `RAW_recipes.csv`. Our goal is to find categories or groupings of recipes from this corpus based on their names. 

**Your tasks:**

- Download [`RAW_recipes.csv`](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv) and put it under the `data` directory in the lab folder. As usual, do not push the CSV in your repository. 
- Run the code below. The dataset is quite large, and in this assignment, for speed, you will work with a sample of the dataset. The function `get_recipes_sample` below carries out some preliminary preprocessing and returns a sample of the recipes with most frequent tags. 

> *Note: Depending upon the capacity of your computer, feel free to increase or decrease the size of this sample by changing the value for `n_tags`. If you decide to go with a different value of `n_tags`, state it clearly in Exercise 2.1 so that the grader knows about it.* 

In [ ]:
orig_recipes_df = pd.read_csv("data/RAW_recipes.csv")
orig_recipes_df.shape

In [ ]:
def get_recipes_sample(orig_recipes_df, n_tags=300, min_len=5):
    orig_recipes_df = orig_recipes_df.dropna()  # Remove rows with NaNs.    
    orig_recipes_df = orig_recipes_df.drop_duplicates(
        "name"
    )  # Remove rows with duplicate names.
    orig_recipes_df = orig_recipes_df[orig_recipes_df["name"].apply(len) >= min_len] # Remove rows where recipe names are too short (< 5 characters).
    first_n = orig_recipes_df["tags"].value_counts()[0:n_tags].index.tolist() # Only consider the rows where tags are one of the most frequent n tags.
    recipes_df = orig_recipes_df[orig_recipes_df["tags"].isin(first_n)]
    return recipes_df

In [ ]:
recipes_df = get_recipes_sample(orig_recipes_df)
recipes_df.shape

In [ ]:
recipes_df["name"]

<br><br>

**In the rest of the lab, use `recipes_df` above, which is a subset of the original dataset.** 

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Longest and shorted recipe names 
rubric={accuracy}

**Your tasks:**

1. Print the shortest and longest recipe names (in terms of number of characters) from `recipes_df`. 

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 2

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Word cloud
rubric={accuracy}

**Your tasks:**
1. Create a word cloud for the recipe names. You can use [the `wordcloud` package](https://github.com/amueller/word_cloud), which is available in the course environment. 

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 2

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Representing recipe names
rubric={reasoning}

Before clustering recipe names, we need to encode them. We will encode them using sentence embedding representation.

**Your tasks:**
1. Similar to Exercise 1, create sentence embedding representation of recipe names (`name` column in `recipes_df`). In the rest of the lab, we'll stick to this representation of recipe names.

> *Be patient. This might take a while to run.*

<div class="alert alert-warning">

Solution_2.3
    
</div>

_Points:_ 2

In [ ]:
...

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 3: Clustering recipe names
<hr>

In this exercise you'll cluster recipe names with some of the clustering algorithms we have seen in class. This will also involve making some attempts to pick reasonable hyperparameter values for each clustering method based on the quality of the resulting clusters. For example, for KMeans, you need to specify the number of clusters in advance, which is often challenging on real-world datasets. For DBSCAN, you need to pick appropriate `eps` and `min_samples`. For hierarchical clustering, you need to pick a suitable linkage criterion, distance metric, and prune the tree so that it's possible to visualize and interpret it. 

Here are some methods which may help you with picking reasonable values for the hyperparameters. 
- Visualize the Elbow plot (KMeans). 
- Visualize Silhouette plots. 
- Visualize resulting clusters using `plot_umap_clusters` function from Exercise 1. 
- Sample some recipes from each cluster, manually inspect whether there are coherent semantic themes. (For this, you may use the function `print_clusters` given below.) 
        
> You may use the [`yellowbrick`](https://www.scikit-yb.org/en/latest/) package for visualizing the Elbow plot and the Silhouette plots.

**Note that the process of picking reasonable hyperparameter values will be exploratory, iterative, and will involve manual inspection and judgment, as there is no ground truth to verify how well the model is doing. In your solutions, please do not include everything you try. Only present the results of the most informative trials. Add a narrative to your answer so that it's easy for the grader to follow your choices and reasoning.** 

In [ ]:
def print_clusters(recipes_df, cluster_labels, n_recipes=10, replace=False):
    """
    Given recipes_df containing recipe names and cluster assignment (labels), 
    sample and print n_recipes recipes per cluster. 

    Parameters
    -----------
    recipe_df : pandas dataframe 
        recipes dataframe containing recipe names in the "name" column
    cluster_labels : ndarray or a list
        cluster labels for each row in recipes_df 
    n_recipes : int
        number of examples to sample from each cluster
    replace: bool
        replace flag to pass to the sampling of recipe names

    Returns
    -----------
    None
    """    
    
    grouped = (
        pd.DataFrame(
            {
                "name": recipes_df["name"],
                "cluster_label": cluster_labels,
            }
        )
        .sort_values("cluster_label")
        .groupby("cluster_label")    
    )
    
    for name, group in grouped:
        print(f"Cluster {name}")        
        print(("----------").format(""))        
        print("\n".join(group.sample(n_recipes)['name'].tolist()))
        print("\n\n")

<br><br>

<!-- BEGIN QUESTION -->

### 3.1 K-Means
rubric={reasoning}

**Your tasks:**

1. Cluster recipe titles using KMeans. Make some attempts to determine the optimal number of clusters. 
2. Pick one or two best models and justify your choice. 

<div class="alert alert-warning">

Solution_3.1
    
</div>

_Points:_ 5

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 3.2 Gaussian mixture models 
rubric={reasoning}

**Your tasks:**

1. Cluster recipes using Gaussian mixture models. Make some attempts to determine the optimal number of clusters. 
2. Find a recipe where your intuitions do not match with the cluster assignment and show prediction probabilities for that recipe. Briefly discuss your observations.    

<div class="alert alert-warning">

Solution_3.2
    
</div>

_Points:_ 5

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 3.3 DBSCAN
rubric={reasoning}

**Your tasks:**

1. Cluster recipe names using `DBSCAN` with `metric="cosine"`. Make some attempts to tune the  hyperparameters `eps` and `min_samples`. 

<div class="alert alert-warning">

Solution_3.3
    
</div>

_Points:_ 5

_Type your answer here, replacing this text._

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 3.4 Hierarchical clustering
rubric={reasoning}

**Your tasks:**

1. Try hierarchical clustering with `metric="cosine"` on this problem. Show a dendrogram by using a suitable truncation method. 
2. Create flat clusters by cutting the tree at the appropriate level. 

> *Note: Try orientation="left" of `dendrogram` for better readability of the dendrogram.*

<div class="alert alert-warning">

Solution_3.4
    
</div>

_Points:_ 5

_Type your answer here, replacing this text._

In [ ]:
...

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 4: Manual interpretation of clusters
<hr>

One of the most important steps in clustering is manual interpretation of clusters. In this exercise, you will consider the finalized models from the previous exercise, examine some samples form clusters produced by these models, and try to find semantic themes in the clusters. 

<!-- BEGIN QUESTION -->

### 4.1 Sampling recipe names from clusters
rubric={accuracy}

**Your tasks:**

1. For each of the clustering methods you tried in Exercise 3, pick reasonable hyperparameter values based on your observations and show samples from each cluster for each method using the `print_clusters` methods above. 

<div class="alert alert-warning">

Solution_4.1
    
</div>

_Points:_ 4

In [ ]:
# Print sample recipes from the clusters created by K-Means

...

In [ ]:
# Print sample recipes from the clusters created by GMMs

...

In [ ]:
# Print sample recipes from the clusters created by DBSCAN

...

In [ ]:
# Print sample recipes from the clusters created by hierarchical clustering

...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 4.2 Manual interpretation of clusters
rubric={reasoning}

**Your tasks:**

Discuss clustering results based on the sampled recipes above. 
1. Label the topics/themes you see in the clusters created by different clustering methods.  
2. Do you see a common theme across clusters created by different clustering methods? Do you see any differences between the clusters created by different clustering methods? 

<div class="alert alert-warning">

Solution_4.2
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 4.3 Examining other attributes in the data for each cluster 
rubric={accuracy}

Recall that we have other attributes such as `minutes`, `n_steps`, `n_ingredients` in our `recipes_df`. 

**Your tasks:**

1. Pick your favourite clustering model from the previous exercises. For each cluster given by this model, show average of `minutes`, `n_steps`, `n_ingredients`. 
2. Do you see any striking differences in the values of these features between clusters? Briefly discuss.  

<div class="alert alert-warning">

Solution_4.3
    
</div>

_Points:_ 3

_Type your answer here, replacing this text._

In [ ]:
...

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 5: Food for thought
<hr>

Similar to DSCI 571, each lab will have a few challenging questions. These are usually low-risk questions and will contribute to maximum 5% of the lab grade. The main purpose here is to challenge yourself, dig deeper in a particular area, and going beyond what we explicitly discussed in the class. When you start working on labs, attempt all other questions before moving to these challenging questions. If you are running out of time, please skip the challenging questions. 

![](img/eva-game-on.png)

<br><br>

<!-- BEGIN QUESTION -->

### (Challenging) 5.1: Similarity measure for mixed datasets
rubric={reasoning}

Clustering is based on finding similar examples. So using appropriate similarity metric is crucial in order to find meaningful clusters. When the data contains only numeric features you can apply appropriate transformation (e.g., scaling) and find similarity between examples based on Euclidean distances between points. In document clustering with sentence embedding representation we used cosine similarity. But what if the dataset contains different types of features such as numeric, categorical (e.g., postal code), or multi-valued categorical features (e.g., movie genres)? How would you calculate similarity between examples as a single numeric value and apply clustering methods? Suggest some ideas.  

> As a concrete example, you may explore clustering of movies from [the IMDB Movies Dataset](https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows). 

<div class="alert alert-warning">

Solution_5.1
    
</div>

_Points:_ 1

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### (Challenging) 5.2: Vector Quantization
rubric={reasoning}

One more application of clustering is _vector quantization_, where we find a prototype point for each cluster and replace points in the cluster by their prototype. If our inputs are images, vector quantization gives us a rudimentary image compression algorithm.

We will implement image quantization by filling in the `quantize` and `dequantize` functions below. The `quantize` function should take in an image,  and using the pixels as examples and the 3 colour channels as features, run KMeans clustering on the data with $2^b$ clusters for some hyperparameter $b$. The code should store the cluster means and return the cluster assignments. The `dequantize` function should return a version of the image (the same size as the original) where each pixel's original colour is replaced with the nearest prototype colour.

To understand why this is compression, consider the original image space. Say the image can take on the values $0,1,\ldots,255$ in each colour channel. Since $2^8=256$ this means we need 8 bits to represent each colour channel, for a total of 24 bits per pixel. Using our method, we are restricting each pixel to only take on one of $2^b$ colour values. In other words, we are compressing each pixel from a 24-bit colour representation to a $b$-bit colour representation by picking the $2^b$ prototype colours that are "most representative" given the content of the image. 

**Your tasks:**

1. Complete the `quantize` and `dequantize` functions below.
2. Run the code on an image of your choosing. Display the results for a few different values of $b$.

> *Note 1: If you actually try saving this as a file, you won't see the file size being what you expected, because Python won't know to allocate exactly $b$ bits per element of `quantized_img`, and will instead probably store them as 32-bit integers if you don't specify otherwise. But if one wanted to work harder, it would be theoretically possible to store the elements with $b$ bits per pixel. Also, all this is before any additional lossless compression.*

> *Note 2: This is not how image compression systems like JPEG actually work. They use something similar to the Fourier transform, followed by a (simpler) quantization, followed by lossless compression.*

<div class="alert alert-warning">

Solution_5.2
    
</div>

_Points:_ 2

In [ ]:
import os

from matplotlib.pyplot import imread, imshow
from sklearn.cluster import KMeans

In [ ]:
img = imread(os.path.join("img/eva-happy-saturday.jpg"))
imshow(img)
plt.title("original image")
plt.show()

In [ ]:
def quantize(img, b):
    """
    Quantizes an image into 2^b clusters

    Parameters
    ----------
    img : a (H,W,3) numpy array
      the image to be processed
    b   : int
      the desired number of bits

    Returns
    -------
    quantized_img : a (H,W) numpy array containing cluster indices
    colours       : a (2^b, 3) numpy array, each row is a colour
    """

    H, W, _ = img.shape
    model = KMeans(n_clusters=2 ** b)

    ### YOUR CODE
    ...

    return quantized_img, model.cluster_centers_.astype("uint8")


def dequantize(quantized_img, colours):
    H, W = quantized_img.shape
    img = np.zeros((H, W, 3), dtype="uint8")

    # YOUR CODE: fill in the values of `img` here

    ...

    return img

In [ ]:
img.shape

In [ ]:
img.reshape(398 * 398, 3).shape

In [ ]:
b = 3
compressed, colours = quantize(img, b=b)
recon = dequantize(compressed, colours)
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(img)
ax[0].set_title("original image")
ax[1].imshow(recon)
ax[1].set_title(f"reconstructed image (b = {b})");

In [ ]:
plt.imshow(colours[None])
plt.title("colours learned")
plt.xticks([])
plt.yticks([])
plt.show()

<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top. 

Well done!! Have a great weekend! 

In [ ]:
from IPython.display import Image

Image("img/eva-well-done.png")